In [1]:
import pandas as pd
import yfinance as yf
import altair as alt
import project_functions2 as pf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import r2_score
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
bitcoin = yf.Ticker('BTC-USD')

In [3]:
history = bitcoin.history(period='max', interval='1d')

In [4]:
history.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800,0,0
2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200,0,0
2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700,0,0
2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600,0,0
2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100,0,0


In [5]:
def bitcoin_neural_net(history_df, split_time):
    scaler = MinMaxScaler()
    X = history_df.iloc[:,:-1]
    y = history_df.iloc[:,-1:]
    
    # Does train/Test Split on last year
    # Change the -50 to a differnt value to change split point
    split_mark = int(len(history_df)-split_time)
    X_train = X.head(split_mark)
    X_test = X.tail(len(history_df) - split_mark)
    y_train = y.head(split_mark)
    y_test = y.tail(len(history_df) - split_mark)

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    early_stopping = EarlyStopping(monitor='loss',
                               patience=3, restore_best_weights=True)
    bitcoin_nn = Sequential()
    bitcoin_nn.add(Dense(units=200, input_dim=X_train_scaled.shape[1], activation='relu'))
    bitcoin_nn.add(Dense(units=200, activation='relu'))
    bitcoin_nn.add(Dense(units=1, activation='relu'))
    bitcoin_nn.compile(optimizer = 'adam', loss = 'mean_squared_error')
    bitcoin_nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping])
    
    bitcoin_nn_pred = bitcoin_nn.predict(X_test_scaled)
    bitcoin_nn_train_pred = bitcoin_nn.predict(X_train_scaled)
    
    
    # Plots Results

    train_score = r2_score(y_train, bitcoin_nn_train_pred)
    print("Training R2 Score: " + str(train_score))
    print()

    columns = []
    columns.append('Bitcoin Price')
    columns.append('Bitcoin Prediction')
    
    results_df = pd.DataFrame(columns=columns, 
                              index=X_test.index)

    results_df[columns[0]] = y_test
    results_df[columns[1]] = bitcoin_nn_pred
        
    results_df.reset_index(inplace=True)
    results_df = results_df.melt('Date', var_name='Pred or Price', value_name='Price')
    # Plots Results
    line_plot = alt.Chart(results_df).mark_line().encode(
        x = 'Date',
        y = 'Price',
        color = 'Pred or Price'
    )
    
    model_score = r2_score(y_test.dropna(), bitcoin_nn_pred[:len(y_test.dropna())])
    print("R2 Score: " + str(model_score))
    
    return line_plot

In [6]:
history.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
history = pf.future_close_setup(history, 1)

bitcoin_neural_net(history, 255)

Epoch 1/100
68/68 [==============================] - 1s 1ms/step - loss: 35164286.1159
Epoch 2/100
68/68 [==============================] - 0s 1ms/step - loss: 34214690.9855
Epoch 3/100
68/68 [==============================] - 0s 2ms/step - loss: 30215623.2754
Epoch 4/100
68/68 [==============================] - 0s 2ms/step - loss: 19980383.0435
Epoch 5/100
68/68 [==============================] - 0s 1ms/step - loss: 8853484.0072
Epoch 6/100
68/68 [==============================] - 0s 1ms/step - loss: 3595444.8007
Epoch 7/100
68/68 [==============================] - 0s 1ms/step - loss: 2505347.2210
Epoch 8/100
68/68 [==============================] - 0s 1ms/step - loss: 2022154.3841
Epoch 9/100
68/68 [==============================] - 0s 900us/step - loss: 1577110.2337
Epoch 10/100
68/68 [==============================] - 0s 613us/step - loss: 1325051.7627
Epoch 11/100
68/68 [==============================] - 0s 505us/step - loss: 970799.6458
Epoch 12/100
68/68 [=======================

alt.Chart(...)